# 4. Regularization

In [ ]:
%matplotlib inline
import warnings;warnings.filterwarnings('ignore');import matplotlib.pyplot as plt;import numpy as np;import pandas as pd;import seaborn as sns;sns.set_context("notebook", font_scale=1.4);sns.set_style("whitegrid");import imp;compomics_import = imp.load_source('compomics_import', '../compomics_import.py');from IPython.core.display import HTML;css_file = '../my.css';HTML(open(css_file, "r").read())

Real world data sets suffer from many shortcomings. The most important one is the presence of **noise**, which manifests itself at different levels:

- **noise in the feature values**: the true (underlying) feature values differ from the observed values in the data sets. This is typically the case happens when feature values are obtained from measuring devices such as microarrays or mass spectrometers.

- **label noise**: the true underlying target or label for a data point is different from the one observed in the data set. This happens for instance when a data set is manually labeled by a field expert (e.g. for the recognition of malignant melanoma). The expert can be wrong for some of the samples. Also, different experts may judge samples differently. 

- **redundant** or **irrelevant** features: redundant features are those which provide no more information than other features in the data set, and irrelevant features provide no useful information for the prediction task.

In the **presence of noise the model should not try to fit the data perfectly**. If it does it will be fitting the noise (as we have seen in the chapter about model selection). Similarly, **if we are not sure all features are relevant, the model should not try to use all features**. 

Let's illustrate this with a simple polynomial regression example. We will generate a data set that contains 30 data points that were randomly sampled from the function $f(x)=\cos(1.5\pi x)$. Next we add random noise to the target and make polynomial data transformations of varying degrees. Finally we plot the resulting model fits:

In [ ]:
from sklearn.linear_model import LinearRegression

np.random.seed(0)

n_samples = 30

def true_fun(X):
    return np.cos(1.5 * np.pi * X)

dataset_reg = pd.DataFrame()
dataset_reg['x1'] = np.sort(np.random.rand(n_samples))
dataset_reg['y'] = true_fun(dataset_reg['x1']) + np.random.randn(n_samples) * 0.1 #adding noise

model_reg = LinearRegression()
compomics_import.plot_regression_degree(dataset_reg,model_reg,true_fun,[2, 4, 12, 18])

For each plot the polynomial degree of the model as well as $R^2$ is shown in the title. 

We observe how $R^2$ becomes better when we increase the degree of the polynomial, i.e. when we increase the complexity of the model. However we can also observe very clearly how the complex models start to fit the noise in the data set and deviate strongly from the true function that generated the data (green curve).

In the chapter about model selection we have seen how we can avoid overfitting the degree $d$: use $k$-fold CV to create validation sets and decide on the opimal value for $d$ using the predictions on these validation sets (and ultimately also on the test set). 

By searching for the optimal value for $d$ using CV we are actually trying to control the **complexity** of the model such that it performs well on both the train **and** the validation set. As the validation set mimics the test set we can expect best generalization performance for the optimal value of $d$ obtained on this validation set.

Controlling the complexity of the model applies to all machine learning algorithms. Consider for instance the following data set separated into two classes by the logistic regression algorithm:

In [ ]:
from sklearn.linear_model import LogisticRegression

dataset_clf = pd.read_csv("dataset_clf.csv")

X = dataset_clf.copy()
y = X.pop('y')
model_clf = LogisticRegression(C=10000)
model_clf.fit(X,y)

plt.figure(figsize=(6,6))
compomics_import.plot_decision_boundary(model_clf,X,y)
plt.show()

This data can never be separated with 100% accuracy using a linear model. In this case the accuracy on the train set is 94%. However, we can again add polynomial transformations of the features to obtain better classification accuracy on the train set. As we have two features $x_1$ and $x_2$ the polynomial features of degree 2 added to the data set are $x_1^2$, $x_1x_2$ and $x_2^2$. We can compute these polynomial features easily with the `PolynomialFeatures` module in the `sklearn.preprocessing` package:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler

feature_scaler = MinMaxScaler()

clf = LogisticRegression(C=1000000)
polynomial_features = PolynomialFeatures(degree=7)
model_pipeline = Pipeline([("polynomial_features", polynomial_features),
                           ("MinMaxScaler", feature_scaler),
                         ("logistic_regression", clf)])

model_pipeline.fit(X,y)

plt.figure(figsize=(6,6))
compomics_import.plot_decision_boundary(model_pipeline,X,y)
plt.show()

In this case the train set accuracy is 95%. However, as with the example of regression discussed above we can again observe that this fit with a polynomial of degree 7 might be far from the true model that generated the data. Again we seem to overfit the possible noise in the data set.

The idea behind **model regularization** is to find a balance between the complexity of the model (e.g. the degree of the polynomial) and the ability of the model to fit the data accurately. Indeed the linear fit shown for the regression task discussed above lacks the complexity (or flexibility) to model the data correctly. 

A more complex model of degree 4 is required. However, for even more complex models such as degree 18 the model starts to overfit the noise in the data. So this model is too complex. This means that we have to **regulate the complexity of the model**: make sure the complexity is sufficient to learn the true model that generated the data, but not more complex than that.

Mathematically we can formalize **regularization** as an additional term to the cost function of the learning algorithm. 

For regularized linear regression we use the cost function

$$ J(\theta)=\frac{1}{2n}\sum\limits_{i=1}^n(f(x^{(i)},\theta) - y^{(i)})^2 + \lambda \sum\limits_{j=1}^{m}\theta_j^2.$$

The term added to the cost function of linear regression is $\lambda \sum\limits_{j=1}^{n}\theta_j^2$, i.e. the sum of the squared weights multiplied by $\lambda$. 

Minimizing the cost function $J(\theta)$ thus means minimizing the magnitude of the errors made on the data set while minimizing also the complexity of the model (i.e. small values for $\theta$, preferably zero). The contribution of the model complexity to the cost function is then controlled by **hyperparameter** $\lambda\geq0$, which is typically optimized using cross-validation. For **regularized linear regression** (or **Ridge regression**) the updates computed during the gradient descent iterations are 

$$\theta_0:=\theta_0-\alpha\frac{1}{n}\sum\limits_{i=1}^n(f(x^{(i)},\theta)-y^{(i)})x_0^{(i)},$$

and for $j=1 \ldots m$

$$\theta_j:=\theta_j-\alpha\frac{1}{n}\sum\limits_{i=1}^n(f(x^{(i)},\theta)-y^{(i)})x_j^{(i)}-\frac{\lambda}{m}\theta_j,$$

with 

$$f(x,\theta)=\theta_0x_0 + \theta_1x_1 + \theta_2x_2+...+\theta_mx_m.$$

For **regularized logistic regression** the cost function becomes

$$J(\theta)=-[\frac{1}{n}\sum\limits_{i=1}^ny^{(i)}log(f(x^{(i)},\theta))+(1-y^{(i)})log(1- f(x^{(i)},\theta))]+\frac{\lambda}{2m}\sum\limits_{j=1}^{m}\theta^2,$$

where $\lambda\geq0$ is again a model parameter that controls the balance between model training set accuracy and model complexity and is again set by the user. The updates computed during the gradient descent iterations are now

$$\theta_0:=\theta_0-\alpha\frac{1}{n}\sum\limits_{i=1}^n(f(x^{(i)},\theta)-y^{(i)})x_0^{(i)},$$

and for $j=1 \ldots m$

$$\theta_j:=\theta_j-\alpha\frac{1}{n}\sum\limits_{i=1}^n(f(x^{(i)},\theta)-y^{(i)})x_j^{(i)}-\frac{\lambda}{m}\theta_j,$$

with

$$f(x,\theta)=g(\theta_0x_0 + \theta_1x_1 + \theta_2x_2+...+\theta_mx_m).$$

The regularized linear regression algorithm is implemented as the `Ridge` module in scikit-learn. Here we create a Ridge model with $\lambda=0.001$ (note the $\lambda$ is provided as function parameter *alpha*).

In [ ]:
from sklearn.linear_model import Ridge

model_ridge = Ridge(alpha=0.001)

Now we can copy the code we used for the regression data set discussed above and replace the `LinearRegression` object by the `Ridge` object.

In [ ]:
compomics_import.plot_regression_degree(dataset_reg,model_ridge,true_fun,[2, 4, 12, 18])

Observe how the complexity of the model is controlled for the larger polynomial degrees. Let's take a look at the optimal model parameter values $\theta$ for the polynomial model of degree 12 when $\lambda=0.001$:

In [ ]:
degree = 12

X = dataset_reg.copy()
y = X.pop('y')
for j in range(2,degree+1):
    X['x1^'+str(j)] = X['x1']**j

model_ridge.fit(X,y)
coefs = model_ridge.coef_

model_parameters = pd.DataFrame()
model_parameters['theta'] = range(1,len(coefs)+1)
model_parameters['value of theta'] = coefs

plt.figure(figsize=(14,4))
sns.barplot(x='theta',y='value of theta',data=model_parameters,color='g')
plt.show()

If we set $\lambda=0$, i.e. we remove the regularization term, we get the overfitting again:

In [ ]:
model_ridge = Ridge(alpha=0)

compomics_import.plot_regression_degree(dataset_reg,model_ridge,true_fun,[2, 4, 12, 18])

model_ridge.fit(X,y)
coefs = model_ridge.coef_

model_parameters = pd.DataFrame()
model_parameters['theta'] = range(1,len(coefs)+1)
model_parameters['value of theta'] = coefs

plt.figure(figsize=(14,4))
sns.barplot(x='theta',y='value of theta',data=model_parameters,color='g')
plt.show()

We can see that the values for $\theta$ are much larger now and much more weight is assigned to the higher degree polynomial features.  This made it possible to fit more complex models that are able to fit the noise well. Regularization is thus closely related to the nature of the noise in the data set. Regularization is also related to the relative size of the dataset. Data sets with many features and few samples are common in machine learning and are easy to overfit. Regularization helps significantly to deal with these **sparse high-dimensional** data sets. 

For now we have chosen $\lambda=0.001$ as a good regulator. Let's consider other values for $\lambda$ and see how they affect the model parameters and the fit. 

We fit different Ridge models (with different values for $\lambda$) to the data and plot the optimal values for $\theta$ against $\lambda$. Each model parameter $\theta_j$ is represented by a color: 

In [ ]:
n_lambda = 40
lambdas = np.logspace(-6, -10, n_lambda)

model_ridge = Ridge()

coefs = []
for l in lambdas:
    model_ridge.set_params(alpha=l)
    model_ridge.fit(X, y)
    coefs.append(model_ridge.coef_)

plt.figure(figsize=(12,6))
compomics_import.plot_coefs(lambdas,coefs)
plt.show()    

The plot shows how larger values for $\lambda$ constrain the values the model parameters can have.

For regularized logistic regression we reconsider the artificial data set we created above and fit a regularized logistic regression model to the data augmented with polynomial features of degree 7. We use the `LogisticRegression` module from scikit-learn that is in fact already an implementation of regularized logistic regression controlled by the function parameter $C$. However in the implementation $C$ = $1/\lambda$ and can be seen as the cost paid for a misclassification. Larger values for $C$ will allow for more complex models. This is why for the example above we set $C$=100000 to allow for maximum model complexity, which mimics unregularized logistic regression. In the example shown below we decrease $C$ to 1, i.e. we lower the complexity of the model:

In [ ]:
X = dataset_clf.copy()
y = X.pop('y')

clf = LogisticRegression(C=1)
polynomial_features = PolynomialFeatures(degree=7)
model_pipeline = Pipeline([("polynomial_features", polynomial_features),
                           ("MinMaxScaler", feature_scaler),
                         ("logistic_regression", clf)])

model_pipeline.fit(X,y)

plt.figure(figsize=(6,6))
compomics_import.plot_decision_boundary(model_pipeline,X,y)
plt.show()

This model does not try to fit the possible noise as much anymore. 

We now have a means to control the complexity of the model learned from the train set by setting the value of a hyperparameter that controls the **balance between train set accuracy and model complexity**. We want to find the value for this hyperparameter that allows for training a model that performs best on unseen external data. We can (or should) thus apply a cross-validation loop to compute the optimal value for the hyperparameter using a validation set as we have seen in the chapter about model selection and validation. 

In scikit-learn there is a package `sklearn.grid_search` that contains the `GridSearchCV` module. This module performs cross-validation to find the best hyperparameter combination from a set of candidate hyperparamter values with a minimal of coding:

In [ ]:
from sklearn.grid_search import GridSearchCV

search_space = np.logspace(-10, 18, 10, base=2)

params = dict(logistic_regression__C=search_space)
grid_search = GridSearchCV(model_pipeline, param_grid=params)
grid_search.fit(X, y)

To view the result of the hyperparameter optimization procedure we can use the following code:

In [ ]:
for params, mean_score, scores in grid_search.grid_scores_:
    print("{:.3f} (+/-{:.3f}) for {}".format(mean_score, scores.std() * 2, params))

We see that the CV performance is optimal for values $C$ arround 50. In this case the average accuracy on the validation sets is 94.2%. However, we cannot consider this accuracy as a performance evaluation of a model with $C=50$. To estimate the performance on unseen external data we have to use an independent test set. However, as the data set is small, leaving out more data might produce unstable results that differ depending on which data points are selected for the test set. 

To solve this we use cross-validation to create the test set as well. By combining the cross_val_predict() and the GridSearchCV() methods of scikit-learn we can create what is know as a **nested cross-validation** loop to compute the predictions on a test set that contains all the data points in the data set:

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.grid_search import GridSearchCV

params = dict(logistic_regression__C=search_space)
grid_search = GridSearchCV(model_pipeline, param_grid=params)

cv_predictions = cross_val_predict(grid_search, X, y, method="predict_proba")

From these predictions we can now compute an estimate of the generalization performance of our optimal model. Here for instance we compute the AUC:

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y, cv_predictions[:,1])
print(metrics.auc(fpr, tpr))